In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random

  Activating environment at `~/Documents/planar_quadruped_landing/Project.toml`


In [2]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("utils.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl")

WebIO._IJuliaInit()

reference_trajectory

## Reference Trajectory

## Problem Definition

In [3]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Discretization
tf = 0.6
dt = 0.02
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N)
t_trans = 0.2

n = 14
m = 4

# Initial Conditions
# currently, we assume the initial mode ID is 1
# xinit = [-0.4;0.6;-3.1415/9;        0.0;0.0; -1.0;0.2; 0.0;-0.5;0.0; 0.0;0.0; 0.0;-0.5]
# xterm = [-lb/2;sqrt(l1^2+l2^2);0.0; 0.0;0.0; -lb;0.0;  0.0;0.0;0.0;  0.0;0.0; 0.0;0.0]
xinit = zeros(n)
xinit[1] = -lb/2                  # xb
xinit[2] = sqrt(l1^2+l2^2) + 0.05 # yb
xinit[3] = 10 * pi / 180          # theta
xinit[6] = -lb                    # x2
xinit[7] = 0.05                   # y2
xinit[9] = -3.0                   # yb_dot
xinit[12] = -3.0                  # y2_dot

xterm = zeros(n)
xterm[1] = -lb/2           # xb
xterm[2] = sqrt(l1^2+l2^2) # yb
xterm[6] = -lb             # x2

init_mode = 1

# Reference Trajectory
Xref, Uref = reference_trajectory(model, times, t_trans, xinit, xterm, init_mode)

# Objective
Random.seed!(1)
Q = Diagonal([1.0;1.0;1.0; 1.0;1.0; 1.0;1.0; 1.0;1.0;1.0; 1.0;1.0; 1.0;1.0])
R = Diagonal(fill(1e-3,4))
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q,R,Xref[k],Uref[k])
end
push!(obj, LQRCost(Qf, R*0, Xref[N], Uref[1]))

# Define the NLP
nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end]);

## Solve

In [4]:
nlp.cinds

6-element Vector{UnitRange{Int64}}:
 1:14
 15:28
 29:448
 449:479
 480:500
 501:562

In [5]:
# Initial guess
xtransit = zeros(n)
xtransit[1] = -lb/2 # xb
xtransit[2] = l1    # yb
xtransit[3] = 0     # theta
xtransit[6] = -lb   # x2

# Random.seed!(1)
# Uguess = [u + 0.1*randn(length(u)) for u in Uref]
# Xguess = [x + 0.1*randn(length(x)) for x in Xref]

Uguess = [zeros(length(u)) for u in Uref]
Xguess = [zeros(length(x)) for x in Xref]

timesteps_phase_1 = nlp.k_trans

for i = 1:timesteps_phase_1
    Xguess[i] = xinit + (xtransit - xinit)/timesteps_phase_1 * i
end

# display((xtransit - xinit)/timesteps_phase_1)

In [6]:
Z0 = packZ(nlp, Xguess, Uguess);
# nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end], use_sparse_jacobian=false);

# Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)
Z_sol, solver = solve(Z0, nlp, c_tol=1e-4, tol=1e-2)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   277000
Number of nonzeros in inequality constraint Jacobian.:    34348
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      554
                     variables with only lower bounds:       93
                variables with lower and upper bounds:    

([-0.24999886233555993, 0.40355347140855274, 0.1745322415613208, -6.752545970290298e-7, 2.86787438963469e-5, -0.5000007118113683, 0.05000007611116693, 4.4096971340097415e-7, -0.5325804902340591, -1.6750618778706061e-7  …  1.3553567577807633e-5, -0.4999969060422728, 1.3114181808407246e-5, 3.344937447606481e-6, -0.0032652996840109136, -2.2094692009919486e-7, -1.102245073886979e-7, -8.477527319046585e-8, -2.453276342845897e-7, -8.195551078172023e-7], Ipopt.Optimizer)

In [10]:
Z_sol[1:14] - xinit

14-element Vector{Float64}:
  1.1376644400706848e-6
  8.081527896885987e-8
 -6.836381121477775e-7
 -6.752545970290298e-7
  2.86787438963469e-5
 -7.118113682924587e-7
  7.611116692451958e-8
  4.4096971340097415e-7
  2.467419509765941
 -1.6750618778706061e-7
  8.328879187999032e-8
  4.089983123733987e-6
 -2.9603264147791164e-8
 -1.5539032060057605e-7

In [11]:
Z_sol[end-13:end] - xterm

14-element Vector{Float64}:
 -5.967959064157036e-6
 -2.0148044194145953e-7
  6.766141914025881e-7
  2.744167875907767e-6
  1.3553567577807633e-5
  3.093957727184904e-6
  1.3114181808407246e-5
  3.344937447606481e-6
 -0.0032652996840109136
 -2.2094692009919486e-7
 -1.102245073886979e-7
 -8.477527319046585e-8
 -2.453276342845897e-7
 -8.195551078172023e-7

EXIT: Restoration failed!

EXIT: Converged to a point of local infeasibility. Problem may be infeasible.

In [9]:
# # display(Z_sol)
# using DelimitedFiles
# using CSV
# writedlm( "data.csv",  Z_sol, ',')